In [2]:
import random, urllib
import sys, os
import requests
import json
import datetime as dt

In [ ]:
url = 'https://rda.ucar.edu/cgi-bin/login'
values = {'email' : '$YOUR_EMAIL', 'passwd' : '$YOUR_PASSWORD', 'action' : 'login'}
# Authenticate
ret = requests.post(url,data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)

# Generate GFS forecast data urls

In [3]:
url_base = 'https://rda.ucar.edu/data/ds084.1/'
with open('D:/github/wrf-helper/config/nl.conf','r') as fp:
    nl_dict = json.load(fp)

In [4]:
interval_hours = int(nl_dict['interval_hours'])
if (interval_hours <= 0) or (interval_hours % 3 != 0):
    raise ValueError ('check interval hours setting')
if interval_hours > 384:
    raise ValueError ('interval hours > 384')
date_start = dt.datetime.strptime(nl_dict['date_start'], '%Y-%m-%d_%H:%M:%S')
date_end = dt.datetime.strptime(nl_dict['date_end'], '%Y-%m-%d_%H:%M:%S')

In [5]:
hour1 = date_start.hour
if hour1 < 6:
    tz = '00'
    f0 = hour1 
elif hour1 < 12:
    tz = '06'
    f0 = hour1 - 6
elif hour1 < 18:
    tz = '12'
    f0 = hour1 - 12
elif hour1 < 24:
    tz = '18'
    f0 = hour1 - 18

hour_count = date_end - date_start
print('cycle: ', tz)
print('starting hour: ', f0)

cycle:  00
starting hour:  0


In [6]:
year = date_start.year
ymd = dt.datetime.strftime(date_start, '%Y%m%d')
duration = date_end - date_start
total_hours = int(duration.seconds / 3600)
fc_hours = [i for i in range(f0, f0+total_hours+1, interval_hours)]

In [7]:
url_list = []
for h in fc_hours:
    link = f'{year}/{ymd}/gfs.0p25.{ymd}{tz}.f{h:03d}.grib2'
    url_list.append(os.path.join(url_base, link))

In [8]:
url_list

['https://rda.ucar.edu/data/ds084.1/2017/20170620/gfs.0p25.2017062000.f000.grib2',
 'https://rda.ucar.edu/data/ds084.1/2017/20170620/gfs.0p25.2017062000.f003.grib2',
 'https://rda.ucar.edu/data/ds084.1/2017/20170620/gfs.0p25.2017062000.f006.grib2',
 'https://rda.ucar.edu/data/ds084.1/2017/20170620/gfs.0p25.2017062000.f009.grib2',
 'https://rda.ucar.edu/data/ds084.1/2017/20170620/gfs.0p25.2017062000.f012.grib2']